<a href="https://colab.research.google.com/github/marofmar/60daysofudacity/blob/master/L7_6%2C7%2C8(Day09).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch as th
!pip install syft
import syft as sy

In [0]:
hook = sy.TorchHook(th) 
bob = sy.VirtualWorker(hook, id = 'bob')

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob) 
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:26702357767 -> bob:55118974860]

In [0]:
y

(Wrapper)>[PointerTensor | me:38372826403 -> bob:662688987]

In [0]:
z = x+y

In [0]:
z

(Wrapper)>[PointerTensor | me:32728448973 -> bob:11078853731]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:56678727739 -> bob:63987109027]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1,2,3.,4,5], requires_grad = True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad = True).send(bob)

In [0]:
z = (x+y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:74952275377 -> bob:8502644253]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
# 6x + 8 = 50
x = th.tensor([7.,10,13], requires_grad = True).send(bob)
#x = th.tensor([1,2,3.,4,5], requires_grad = True).send(bob)
y = th.tensor([14., 20, 26], requires_grad = True).send(bob)

In [0]:
z = x+10

In [0]:
z

(Wrapper)>[PointerTensor | me:57939563706 -> bob:71257046830]

In [0]:
z.backward() #error

In [0]:
z

(Wrapper)>[PointerTensor | me:41092139718 -> bob:84670148964]

In [0]:
z.get()

tensor([20., 32., 44.], requires_grad=True)

In [0]:
x.get()

tensor([1., 2., 3.], requires_grad=True)

In [0]:
y.get()

tensor([14., 20., 26.], requires_grad=True)

In [0]:
z = th.add(x,y)

In [0]:
z.backward() #error

In [0]:
input = th.tensor([[1.1,1],[0.,1],[1.,0],[0,0]], requires_grad = True).send(bob)
target = th.tensor([[1.],[1.],[0],[0]], requires_grad = True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad = True).send(bob)

In [0]:
pred = input.mm(weights)

In [0]:
loss =( (pred-target)**2).sum()

In [0]:
pred = input.mm(weights)
loss.backward()

(Wrapper)>[PointerTensor | me:19013013329 -> bob:62681675544]

In [0]:
for i in range(10):

  pred = input.mm(weights)
  loss =( (pred-target)**2).sum()
  pred = input.mm(weights)
  loss.backward()
  weights.data.sub_(weights.grad*0.1)
  weights.grad*=0

  print(loss.get().data)

tensor(0.1412)
tensor(0.0827)
tensor(0.0519)
tensor(0.0331)
tensor(0.0212)
tensor(0.0135)
tensor(0.0087)
tensor(0.0055)
tensor(0.0036)
tensor(0.0023)


# Practice

In [0]:
hook = sy.TorchHook(th) 
alice = sy.VirtualWorker(hook, id = 'alice')

W0707 14:26:15.376127 139936032589696 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
input = th.tensor([[1.,1],[1.,0],[0,1.],[0,0]],requires_grad = True).send('alice')
target = th.tensor([[1.],[1.],[1.],[0]],requires_grad = True).send('alice')

In [0]:

weights = th.tensor([[0.],[0.]], requires_grad = True).send('alice')

In [0]:
pred = input.mm(weights)

In [0]:
loss = ((pred-target)**2).sum()

In [0]:
loss.backward()

(Wrapper)>[PointerTensor | me:96982983816 -> alice:2543271148]

In [0]:
# for i in range(10):

#   pred = input.mm(weights)
#   loss =( (pred-target)**2).sum()
#   #pred = input.mm(weights)
#   loss.backward()
#   weights.data.sub_(weights.grad*0.1)
#   weights.grad*=0

#   print(loss.get().data)
  
for i in range(5):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad*=0 
    print(loss.get().data)

tensor(3.)
tensor(0.4400)
tensor(0.3504)
tensor(0.3361)
tensor(0.3338)


In [0]:
# input, target, weights settled

for i in range(5):
  pred = input.mm(weights)
  loss = ((pred-target)**2).sum()
  loss.backward() 
  weights.data.sub(weights.grad*0.1) 
  weights.grad*=0
  print(loss.get().data)

tensor(0.3334)
tensor(0.3334)
tensor(0.3334)
tensor(0.3334)
tensor(0.3334)
